## 00. 프로젝트 목적
- 본 프로젝트는 Pinecone vector store를 위한 연습 프로젝트입니다.

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

### 필요한 환경변수 로드

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os


# API 키 정보 로드
load_dotenv()

# 환경 변수에서 OpenAI API 키 읽어오기
openai_api_key = os.environ.get('OPENAI_API_KEY')

# 환경 변수에서 Pinecone API 키 읽어오기
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

## 01. PDF to Text
1. PDF 텍스트를 추출하는데 강점을 가진 함수
    - `extract_text_from_pdf`
    - 텍스트가 주를 이루고 있을 때, 사용한다.
2. PDF to Markdown 함수
    - `extract_markdown_from_pdf`
    - 표와 텍스트가 병합하여 사용되고 있을 때 사용하면 유용하다.

In [2]:
import sys
import os

# vector_store 폴더의 상위 디렉토리인 프로젝트 루트를 수동으로 지정합니다.
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from create_dataset.utils import extract_markdown

# PDF 파일에서 마크다운 추출
chunks = extract_markdown.extract_markdown_from_pdf('../create_dataset/source_data/pdf/과목 소개.pdf')

PDF 파일 로드 중: ../create_dataset/source_data/pdf/과목 소개.pdf
Successfully imported LlamaIndex
총 5개의 청크로 나누었습니다.
총 5개의 청크 중 5개의 유효한 청크를 추출했습니다.
총 5개의 청크 중 5개의 병합된 청크를 생성했습니다.
병합으로 0개의 청크가 줄었습니다.


In [3]:
chunks

['--- Chunk 1/5 ---\n### 과목 소개\n\n#\n\n#### ‘공통수학1’과 ‘공통수학2’는 수학에 대한 기초 소양과 학문적 이해를 기반으로 학생 스스로 자신의 적성을 개발하여 창의성을 갖춘 사람으로 성장하기 위해 수학의 여러 영역의 기본적인 내용을 학습하는 과목이다. 특히 ‘공통수학1’은 중학교 ‘변화와 관계’ 영역에서 학습한 다항식, 방정식, 부등식이 심화되고 다양한 유형으로 다루어지며, ‘자료와 가능성’ 영역에서 학습한 경우의 수가 순열과 조합을 활용하는 방법으로 체계화된다.\n#\n\n## 무엇을 배울까요?\n#\n\n### 40 [ 경상남도교육청]\n\n|범주 다항식 방정식과 부등식 지식·이해 경우의 수 행렬|Col2|내용 요소 • 다항식의 연산 • 나머지정리 • 인수분해|\n\n|---|---|---|\n||방정식과 부등식|• 복소수와 이차방정식 • 이차방정식과 이차함수 • 여러 가지 방정식과 부등식|\n||경우의 수|• 합의 법칙과 곱의 법칙 • 순열과 조합|\n||행렬|• 행렬과 그 연산|\n|과정·기능|• 다항식, 방정식과 부등식, 경우의 수, 행렬의 개념, 원리, 법칙이나 자신의 수학적 사고와 전략을 설명하기 • 수학적 절차를 수행하고 계산하기 • 적절한 전략을 사용하여 문제해결하기 • 이차방정식과 이차부등식을 이차함수와 연결하기 • 이차함수의 그래프와 직선의 위치 관계를 판단하기 • 다항식, 방정식과 부등식, 경우의 수, 행렬의 개념, 원리, 법칙, 성질을 탐구하기 • 방정식과 부등식 풀기 • 방정식과 부등식, 경우의 수, 행렬을 실생활과 연결하기 • 식과 그래프, 수학 기호, 행렬 등을 표현하기||\n\n-----\n\n#',
 '--- Chunk 2/5 ---\n## 과목 소개\n\n#\n\n#### ‘공통수학1’과 ‘공통수학2’는 수학에 대한 기초 소양과 학문적 이해를 기반으로 학생 스스로 자신의 적성을 개발하여 창의성을 갖춘 사람으로 성장하기 위해 수학의 여러 영역의 기본적인 내용을 학습하는 과목이다. 특히 ‘공통수

### 02. Text Embedding
- 변환된 텍스트를 벡터로 임베딩
- 텍스트 청크를 Langchain의 Document 객체로 변환하는 과정

In [4]:
from langchain.schema import Document

# Document 리스트로 변환
documents = [Document(page_content=chunk, metadata={"source": "고교학점제"}) for chunk in chunks]

# 각 청크에 'text' 메타데이터 추가
for chunk in documents:
    chunk.metadata['text'] = chunk.page_content

In [5]:
chunk

Document(metadata={'source': '고교학점제', 'text': '--- Chunk 5/5 ---\n## 무엇을 배울까요?\n#\n\n### 44 [ 경상남도교육청]\n\n|범주 지수함수와 로그함수 삼각함수 지식·이해 수열|Col2|내용 요소 • 지수와 로그 • 지수함수와 로그함수|\n\n|---|---|---|\n||삼각함수|• 삼각함수 • 사인법칙과 코사인법칙|\n||수열|• 등차수열과 등비수열 • 수열의 합 • 수학적 귀납법|\n|과정·기능|• 대수의 개념, 원리, 법칙 탐구하기 • 식과 그래프, 수학 기호 등을 비교하고, 표현하기 • 대수의 개념, 원리, 법칙이나 자신의 수학적 사고와 전략 설명하기 • 적절한 전략을 사용하여 문제해결하기 • 대수의 개념, 법칙 활용하기 • 적절한 공학 도구를 선택하여 함수의 그래프 그리고 탐구하기 • 상용로그, 삼각함수를 실생활과 연결하기 • 등차수열과 등비수열의 일반항과 그 합 구하기 • 수학적 귀납법으로 증명하기||\n\n-----'}, page_content='--- Chunk 5/5 ---\n## 무엇을 배울까요?\n#\n\n### 44 [ 경상남도교육청]\n\n|범주 지수함수와 로그함수 삼각함수 지식·이해 수열|Col2|내용 요소 • 지수와 로그 • 지수함수와 로그함수|\n\n|---|---|---|\n||삼각함수|• 삼각함수 • 사인법칙과 코사인법칙|\n||수열|• 등차수열과 등비수열 • 수열의 합 • 수학적 귀납법|\n|과정·기능|• 대수의 개념, 원리, 법칙 탐구하기 • 식과 그래프, 수학 기호 등을 비교하고, 표현하기 • 대수의 개념, 원리, 법칙이나 자신의 수학적 사고와 전략 설명하기 • 적절한 전략을 사용하여 문제해결하기 • 대수의 개념, 법칙 활용하기 • 적절한 공학 도구를 선택하여 함수의 그래프 그리고 탐구하기 • 상용로그, 삼각함수를 실생활과 연결하기 • 등차수열과 등비수열의 일반항과 그 합 구하기 • 수학적 귀납법으로 증명하기||\n\n-----')

In [6]:
from tqdm import tqdm
from openai import OpenAI


# 클라이언트 초기화
client = OpenAI(api_key=openai_api_key)

# 텍스트 → 벡터로 변환
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=[text]
    )
    return response.data[0].embedding


embeddings = []
for chunk in tqdm(chunks, desc="임베딩 중"):
    embeddings.append(get_embedding(chunk))

임베딩 중: 100%|██████████| 5/5 [00:04<00:00,  1.04it/s]


### 03. Pinecone Vector DB
- Index 생성

In [7]:
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone as LangchainPinecone

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=pinecone_api_key)

c:\Users\82106\anaconda3\envs\bearable\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
# Pinecone index 확인
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'bearable-chatbot-7r6utof.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'bearable-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [10]:
# 해당 코드는 한번만 합니다.

# Pinecone 인덱스 생성
index = pc.Index("bearable-chatbot")

# Pinecone 인덱스 이름
index_name = "bearable-chatbot"

# 인덱스 존재 여부 확인 (has_index 대신 list로 체크!)
if index_name not in [index['name'] for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embedding size
        metric="cosine",
        spec={"cloud": "aws", "region": "us-east-1"}
    )

In [19]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

# OpenAI 임베딩 인스턴스 생성
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small',
    openai_api_key=openai_api_key
)

# PineconeVectorStore 생성
pinecone_database = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    index_name="bearable-chatbot"
)

- 삭제할 떄 사용하는 코드

In [18]:
# 네임스페이스 내 모든 벡터 삭제
# index.delete(delete_all=True, namespace="")

{}

### 04. Retriver 
- 리트리버를 이용하여 DB에 있는 문서 호출

1. 유사도 기반 리트리버 (similarity_retriever)

- 코사인 유사도(cosine similarity) 또는 내적(dot product) 등을 기반으로 가장 유사한 문서들을 찾아 반환
- 장점 : 문서 간 의미적 유사성을 비교 → 키워드가 다르더라도 의미가 같으면 잘 작동함

In [20]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# ✅ 기본 Similarity Retriever
similarity_retriever = pinecone_database.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 가장 유사한 top-k 문서 반환
)

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=similarity_retriever,
    return_source_documents=True  # 어떤 chunk에서 답을 가져왔는지도 확인 가능
)

# 질문해보기
query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)

print("Answer:", result["result"])
# print("\nSource documents:")
# for doc in result["source_documents"]:
#     print(doc.metadata, doc.page_content[:100], "...")

C:\Users\82106\AppData\Local\Temp\ipykernel_14896\2947024676.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
C:\Users\82106\AppData\Local\Temp\ipykernel_14896\2947024676.py:21: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)


Answer: 공통수학 1에서 다항식의 종류를 살펴보면, 주로 다음과 같은 다양한 형태의 다항식들을 배우게 됩니다:

1. **일차 다항식**: 변수의 최고차항이 1인 다항식입니다. 예를 들어, \(f(x) = 2x + 3\)과 같은 형태입니다.

2. **이차 다항식**: 변수의 최고차항이 2인 다항식입니다. 예를 들어, \(g(x) = ax^2 + bx + c\) (\(a \neq 0\))와 같은 형태입니다.

3. **삼차 다항식**: 변수의 최고차항이 3인 다항식입니다. 예를 들어, \(h(x) = 4x^3 - 3x^2 + 2x - 1\)과 같은 형태입니다.

4. **고차 다항식**: 변수의 최고차항이 4 이상인 다항식입니다. 예를 들어, \(i(x) = x^4 - 2x^3 + x - 3\)과 같은 형태입니다.

5. **일변수 다항식**: 한 개의 변수만을 포함하는 다항식입니다. 예를 들어, \(j(x) = 3x^2 - 4x + 5\)와 같은 형태입니다.

6. **다변수 다항식**: 두 개 이상의 변수를 포함하는 다항식입니다. 예를 들어, \(k(x, y) = 2x^2 + 3xy + y^2\)와 같은 형태입니다.

이 외에도 다항식을 구성하는 항의 수에 따라 **단항식**(한 개의 항), **이항식**(두 개의 항), **삼항식**(세 개의 항) 등으로 구분할 수 있습니다. 다항식의 개념과 성질, 연산 등은 수학의 기본적이면서도 중요한 부분을 이루며, 고등학교 공통수학에서 다루는 핵심 주제 중 하나입니다.


2. Multi-Query Retriver

- 원리 : 하나의 사용자 질문을 GPT 등의 LLM이 다양한 방식으로 재표현(paraphrase)하여
여러 개의 쿼리를 생성하고, 각각의 쿼리로 벡터 DB에서 검색을 수행한 후,
그 결과를 통합(fusion)하여 최종 관련 문서 집합을 구성함

In [21]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=pinecone_database.as_retriever(search_kwargs={"k": 3}),
    llm=llm
)

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=multi_query_retriever,
    return_source_documents=True  # 어떤 chunk에서 답을 가져왔는지도 확인 가능
)

# 질문해보기
query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)

print("Answer:", result["result"])


Answer: 공통수학 1에서 다루는 다항식의 내용 요소로는 다음과 같은 것들이 있습니다:

- 다항식의 연산
- 나머지 정리
- 인수분해

이 내용들은 다항식을 구성하는 기본적인 개념과 다항식을 다루는 기본적인 방법들을 포함합니다. 다항식의 종류에 대한 구체적인 명시는 없지만, 일반적으로 다항식은 한 변수 또는 여러 변수에 대한 항들의 합으로 이루어진 식을 말하며, 이 과정에서 학생들은 다항식의 기본적인 형태와 연산, 그리고 이를 활용하는 방법을 배우게 됩니다.


3. Max Marginal Relevance

- 원리 : 사용자의 쿼리와 가장 유사한 문서를 찾되, 이미 선택된 문서들과 중복되지 않도록 다양성(diversity)을 고려해서 문서를 선택

In [22]:
mmr_retriever = pinecone_database.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}  # 다양성 확보
)

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=mmr_retriever,
    return_source_documents=True  # 어떤 chunk에서 답을 가져왔는지도 확인 가능
)

# 질문해보기
query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)

print("Answer:", result["result"])


Answer: 공통수학 1에서 다루는 다항식의 내용 요소로는 다음과 같은 것들이 있습니다:

- 다항식의 연산
- 나머지정리
- 인수분해

이러한 내용을 통해 다항식과 관련된 다양한 개념과 연산 방법을 배우게 됩니다.


4. BM25 Retriever
- 원리 : 쿼리와 문서 간의 단어 빈도(TF), 역문서 빈도(IDF), 문서 길이 보정 등을 사용해 정확히 일치하는 단어 중심으로 검색

In [23]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain.chains import RetrievalQA

# 문서 객체 생성
docs = [Document(page_content=chunk) for chunk in chunks]  # chunks는 마크다운 등에서 분리된 텍스트 청크

# BM25 리트리버 생성
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 5

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=bm25_retriever,
    return_source_documents=True
)

query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)
print("Answer:", result["result"])

Answer: 공통수학 1에서 다루는 다항식의 내용 요소로는 다음이 포함됩니다:

- 다항식의 연산
- 나머지정리
- 인수분해

이러한 내용을 통해 학생들은 다항식과 관련된 기본적인 개념과 연산 방법, 나머지정리의 이해 및 활용, 그리고 다항식을 인수분해하는 방법을 학습합니다.


5. Ensemble Retriever (BM25 + Dense)
- 원리  : BM25와 Dense retriever의 결과를 결합하여 키워드 기반과 의미 기반의 장점을 동시에 활용.

In [26]:
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.retrievers import BM25Retriever

# dense (FAISS) retriever
embedding = OpenAIEmbeddings()
faiss = FAISS.from_documents(docs, embedding)
faiss_retriever = faiss.as_retriever(search_kwargs={"k": 3})

# sparse (BM25)
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 3

# 앙상블 리트리버
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=ensemble_retriever,
    return_source_documents=True
)

query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)
print("Answer:", result["result"])

Answer: 공통수학 1에서 다루는 다항식의 내용 요소로는 다음과 같은 것들이 포함됩니다:

- 다항식의 연산
- 나머지정리
- 인수분해

이러한 내용을 통해 학생들은 다항식과 관련된 기본적인 개념과 연산 방법, 나머지정리의 활용, 다항식을 인수분해하는 방법 등을 학습하게 됩니다.


6. Dense Retriever (FAISS)

- 원리 : 문서와 질문을 **벡터화(임베딩)**하고, 코사인 유사도 등의 거리 계산을 통해 가장 가까운 문서를 검색함. 의미 기반 검색의 핵심.

In [27]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# 기존 FAISS retriever
base_retriever = faiss.as_retriever(search_kwargs={"k": 5})

# 임베딩 기반 문서 필터링
compressor = EmbeddingsFilter(embeddings=embedding, similarity_threshold=0.76)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True
)

query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)
print("Answer:", result["result"])

Answer: 공통수학 1에서 다루는 다항식의 내용 요소로는 "다항식의 연산"과 "인수분해"가 있습니다. 이는 다항식을 구성하는 기본적인 개념과 다항식을 간소화하거나 변형하는 방법에 초점을 맞추고 있습니다. 다만, 구체적인 다항식의 종류(예: 일차 다항식, 이차 다항식 등)에 대한 명시는 제공되지 않았습니다. 일반적으로 다항식은 변수의 차수에 따라 분류되며, 이는 공통수학 1 과정에서 학습하는 다항식의 연산 및 인수분해를 통해 더 깊이 이해할 수 있습니다.


7. ParentDocumentRetriever

- 원리 : 큰 문서를 Parent 단위로 저장하고, 검색 시에는 Parent를 먼저 찾은 뒤,
해당 Parent의 하위 청크(child)를 LLM에 전달

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 2. 문서 저장소
docstore = InMemoryStore()

# ✅ 3. ParentDocumentRetriever 생성 시 vectorstore 사용!
parent_retriever = ParentDocumentRetriever(
    vectorstore=pinecone_database,
    docstore=docstore,
    parent_splitter=splitter,
    child_splitter=splitter
)

# 4. 문서 등록
parent_retriever.add_documents(documents)

# 5. QA 체인
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=parent_retriever,
    return_source_documents=True
)

query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)

print("Answer:", result["result"])

Answer: 공통수학 1에서 다항식의 종류를 살펴보면, 주로 다음과 같은 다양한 형태의 다항식들을 배우게 됩니다:

1. **일차 다항식**: 변수의 최고차항이 1인 다항식입니다. 예를 들어, \(f(x) = 2x + 3\)과 같은 형태입니다.

2. **이차 다항식**: 변수의 최고차항이 2인 다항식입니다. 예를 들어, \(g(x) = ax^2 + bx + c\) (\(a \neq 0\))와 같은 형태입니다.

3. **삼차 다항식**: 변수의 최고차항이 3인 다항식입니다. 예를 들어, \(h(x) = 4x^3 - 3x^2 + 2x - 1\)과 같은 형태입니다.

4. **고차 다항식**: 변수의 최고차항이 4 이상인 다항식입니다. 예를 들어, \(i(x) = 2x^4 - x^3 + 3x^2 - x + 5\)와 같은 형태입니다.

5. **일변수 다항식**: 한 개의 변수만을 포함하는 다항식입니다. 예를 들어, \(j(x) = 3x^2 - 2x + 1\)과 같은 형태입니다.

6. **다변수 다항식**: 두 개 이상의 변수를 포함하는 다항식입니다. 예를 들어, \(k(x, y) = x^2 + xy + y^2\)과 같은 형태입니다.

이 외에도 다항식을 구성하는 항의 수에 따라 **단항식**(한 개의 항만을 가진 다항식), **이항식**(두 개의 항을 가진 다항식), **삼항식**(세 개의 항을 가진 다항식) 등으로 구분할 수 있습니다. 다항식의 분류는 그 형태와 변수의 차수에 따라 다양하게 이루어집니다.


8. Self-Query Retriever

- 원리 : LLM이 사용자 질문을 분석해 질문에서 메타데이터 필터 조건(예: 학과="경영대", 학년="3학년")을 추출하고,
이를 기반으로 벡터 검색 + 메타 필터링을 동시에 수행함.

In [32]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(name="source", description="문서 출처", type="string"),
]

self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=pinecone_database,
    document_contents="과목에 대한 요약 콘텐츠",
    metadata_field_info=metadata_field_info,
    verbose=True
)

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=self_query_retriever,
    return_source_documents=True  # 어떤 chunk에서 답을 가져왔는지도 확인 가능
)

# 질문해보기
query = "공통수학 1 다항식에는 무슨 종류가 있어?"
result = qa_chain(query)

print("Answer:", result["result"])


Answer: 공통수학 1에서 다루는 다항식의 내용 요소로는 다음과 같은 것들이 포함됩니다:

- 다항식의 연산
- 나머지정리
- 인수분해

이러한 내용을 통해 다항식과 관련된 다양한 개념과 연산 방법을 배우게 됩니다.


In [34]:
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# LLM 준비
llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

# ✅ 1. 기본 similarity retriever
similarity_retriever = pinecone_database.as_retriever(search_kwargs={"k": 3})

# ✅ 2. MultiQuery retriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=similarity_retriever,
    llm=llm
)

# ✅ 3. SelfQuery retriever
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="문서의 주제 또는 분야 (예: 고교학점제, 진로상담, 독서추천 등)",
        type="text"
    )
]

self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=pinecone_database,
    document_contents="입시 및 행정 문서 내용 요약",
    metadata_field_info=metadata_field_info,
    verbose=True
)

# ✅ 4. 앙상블 리트리버로 결합 (각 리트리버의 weight는 필요시 조정)
ensemble_retriever = EnsembleRetriever(
    retrievers=[similarity_retriever, multi_query_retriever, self_query_retriever],
    weights=[0.3, 0.4, 0.3]
)

# ✅ 5. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=ensemble_retriever,
    return_source_documents=True
)

# ✅ 6. 질의 실행
query = "공통수학 1과 공통수학 2의 차이가 뭐야"
result = qa_chain(query)

print("Answer:", result["result"])

Answer: 공통수학 1과 공통수학 2는 모두 수학의 기초 소양과 학문적 이해를 바탕으로 학생들이 자신의 적성을 개발하고 창의성을 갖춘 사람으로 성장하기 위해 설계된 과목들입니다. 그러나 두 과목은 다루는 내용과 초점에 있어 차이가 있습니다.

### 공통수학 1
- **주요 내용**: 중학교 ‘변화와 관계’ 영역에서 학습한 다항식, 방정식, 부등식을 심화하고 다양한 유형으로 다루며, ‘자료와 가능성’ 영역에서 학습한 경우의 수를 순열과 조합을 활용하는 방법으로 체계화합니다.
- **학습 범주**: 다항식, 방정식과 부등식, 경우의 수, 행렬 등을 포함합니다.

### 공통수학 2
- **주요 내용**: 중학교 ‘변화와 관계’ 영역에서 학습한 함수의 개념을 확장하고, ‘도형과 측정’ 영역에서 학습한 원과 직선을 방정식으로 다룹니다.
- **학습 범주**: 도형의 방정식, 집합과 명제, 함수와 그래프 등을 포함합니다.

요약하자면, **공통수학 1**은 다항식, 방정식, 부등식, 경우의 수, 행렬 등의 기본적인 수학적 개념과 원리를 심화하여 다루는 반면, **공통수학 2**는 함수의 개념 확장, 도형의 방정식, 집합과 명제, 그리고 함수와 그래프에 대한 학습에 더 중점을 둡니다. 따라서 두 과목은 수학의 다양한 영역을 폭넓게 다루면서도 각기 다른 주제에 초점을 맞추고 있습니다.


In [36]:
# ✅ 1. Similarity Retriever (기본 벡터 검색)
similarity_retriever = pinecone_database.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# ✅ 2. MultiQuery Retriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=similarity_retriever,  # 내부적으로 similarity 사용
    llm=llm
)

# ✅ 3. 앙상블 리트리버 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[similarity_retriever, multi_query_retriever],
    weights=[0.5, 0.5]  # 가중치는 필요에 따라 조정 가능
)

# ✅ 4. QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=ensemble_retriever,
    return_source_documents=True
)

# ✅ 5. 질문 테스트
query = "공통수학 1과 공통수학 2의 차이가 뭐야"
result = qa_chain(query)

print("Answer:", result["result"])

Answer: 공통수학 1과 공통수학 2는 모두 수학의 기초 소양과 학문적 이해를 바탕으로 학생들이 자신의 적성을 개발하고 창의성을 갖추도록 설계된 과목이지만, 다루는 내용과 초점에 차이가 있습니다.

### 공통수학 1
- 중학교 ‘변화와 관계’ 영역에서 배운 다항식, 방정식, 부등식을 심화하여 다루며, 다양한 유형으로 접근합니다.
- ‘자료와 가능성’ 영역에서 배운 경우의 수를 순열과 조합을 활용하는 방법으로 체계화합니다.
- 주요 내용 요소로는 다항식의 연산, 나머지 정리, 인수분해, 복소수와 이차방정식, 이차방정식과 이차함수, 여러 가지 방정식과 부등식, 합의 법칙과 곱의 법칙, 순열과 조합, 행렬과 그 연산 등이 포함됩니다.

### 공통수학 2
- 중학교 ‘변화와 관계’ 영역에서 배운 함수의 개념을 확장하고, ‘도형과 측정’ 영역에서 배운 원과 직선을 방정식으로 다룹니다.
- 주요 내용 요소로는 평면좌표, 직선의 방정식, 원의 방정식, 도형의 이동, 집합, 명제, 함수, 유리함수와 무리함수 등이 포함됩니다.

결론적으로, **공통수학 1**은 다항식, 방정식, 부등식, 경우의 수, 행렬 등의 기본적인 수학적 개념과 원리를 심화하여 다루는 반면, **공통수학 2**는 함수의 개념 확장, 도형의 방정식, 집합과 명제, 그리고 함수와 그래프 등 좀 더 고급 수학적 개념과 원리를 다룹니다.
